# Decision Support Layer — Notebook Overview

**Goal:** The Decision Support Layer is designed to support event-driven, low-latency decision making based on continuous acoustic inputs. It interprets structured outputs from the Acoustic ML model using temporal aggregation and state-based logic. The resulting decisions are optimized for fast, human-facing alerts under real-time constraints..


## Architecture Diagrams

### 1) Baseline: Acoustic Detection → Decision Support

![Baseline architecture](docs/diagram-desicion-support-layer-baseline.png)

### 2) Extension: Fusion-Enabled Decision Support

![Fusion extension](docs/diagram-desicion-support-layer-fusion.png)

> **Note:** These images are expected to be placed in the same repository folder as this notebook (or update the paths accordingly).


## README

# Event-Based Acoustic Decision Support Layer

## Overview

This repository implements a **decision-making layer** that operates on top of
**event-based outputs produced by an Acoustic ML model**.

The system **does not process raw audio signals** and does not perform signal processing.
Instead, it consumes a **strict data contract** representing the output of an upstream
Acoustic ML pipeline and transforms it into **time-aware, explainable decision outputs**
suitable for user alerts.

The project is centered around a single core principle:

> **Acoustic ML output is the primary and mandatory input.  
All other modalities are optional and must not block or delay decisions.**

---

## Core Concept

### Primary responsibility

- Define and enforce a **data contract** between:
  - Acoustic ML inference
  - Decision support logic
- Aggregate acoustic detection events over time
- Apply decision rules and state transitions
- Produce structured outputs for alerting and visualization

### Out of scope

- Audio signal processing (DSP)
- Acoustic feature extraction
- ML model training or tuning
- Hardware-level integration

---

## Input Data Contract (Mandatory)

### Primary input: Acoustic ML events

The **main input** to this system is the output of an Acoustic ML model that has already
processed raw microphone data.

This contract is **mandatory** and the system cannot operate without it.

### File structure

```
input/
  sessions/
    session_<timestamp>__<id>/
      acoustic_events.jsonl          ← REQUIRED
      human_interaction.jsonl        ← OPTIONAL
      video_observation.jsonl        ← OPTIONAL
```

Only `acoustic_events.jsonl` is required.  
All other input files may be empty or absent.

---

### Acoustic Events Contract

The acoustic event file represents **event-level ML inference results**, not signals.

Typical fields include:
- `timestamp`
- `drone_detected: true | false`
- `p_drone ∈ [0,1]`
- `proximity: FAR | MID | CLOSE`
- `eta_seconds` (optional)
- `direction / distance` (optional)
- `drone_presence_pattern: single | multiple`
- `environment_context` (acoustically inferred noise conditions)

Schema definition:
```
schemas/acoustic.events.schema.json
```

---

## Optional Fusion Inputs

Optional inputs may provide context but must **never block decisions**.

### Human Interaction (Optional)

- Dictionary-based keyword events
- No free speech recognition
- No raw audio storage
- Privacy-preserving by design

Schema:
```
schemas/human_interaction.schema.json
```

### Video Observation (Optional)

- Coarse observations only
- `detected / not detected`
- Low-frequency updates (~1 Hz)

Schema:
```
schemas/video_observation.schema.json
```

---

## Decision Support Layer

Consumes acoustic (and optional fusion) events and performs:

- Time-aware aggregation
- State machine transitions
- Alert suppression and cooldown handling
- Confidence estimation

### Decision States

- **IDLE** — monitoring only
- **ALERT** — alert issued to the user
- **COOLDOWN** — alert already delivered, no repeated notifications

---

## Decision Output Contract

Decision results are always linked to input sessions.

### Output structure

```
output/
  sessions/
    session_<timestamp>__<id>/
      decision_output.jsonl
```

Schema:
```
schemas/decision_output.schema.json
```

Typical output fields:
- `decision_state`
- `p_drone`
- `confidence_level`
- `alert_triggered`
- `alert_types`
- `proximity`
- `eta_seconds`
- `explanation` (optional)

---

## Handling Multiple Objects

Exact object counting is not performed.

Instead:
```
drone_presence_pattern: single | multiple
```

---

## Design Principles

- Acoustic-first design
- Strict separation of concerns
- Event-based processing
- Time-aware decisions
- Optional fusion inputs
- Explainable outputs
- Privacy-first human interaction

---

## Intended Use

This repository represents a **decision-layer prototype** for:
- architecture validation
- synthetic data replay
- decision logic evaluation
- visualization workflows

---

## Project Status

- Data contracts: defined and stable
- Synthetic sessions: included
- Decision logic: prototype
- Fusion extensions: future work




## Human Interaction Dictionary (Optional Channel)

This channel is a **minimal, structured context** input. It is *not* raw audio, not free text, and does not label objects.

**Fields:**
- `space`: `inside | outside | open_area | forest | near_buildings`
- `mobility`: `stationary | on_foot | in_vehicle`
- `environment`: array of `wind | traffic_noise | quiet | urban_noise`
- `visual_hint`: boolean (human has visual contact with an unidentified object)

**Why:** reduce uncertainty / false positives while preserving privacy.

Example event (JSONL line):

```json
{
  "timestamp_utc": "2026-01-26T21:05:00Z",
  "source": "human_interaction",
  "space": "outside",
  "mobility": "on_foot",
  "environment": ["quiet"],
  "visual_hint": true,
  "confidence": 1.0,
  "privacy": {
    "no_audio_content": true,
    "no_image_data": true,
    "no_object_labeling": true
  }
}
```
